In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from matplotlib import pyplot as plt

In [2]:
data_dir = '2021-07-19'

In [3]:
citation_net = pd.read_csv(f'citation_net_{data_dir}.tsv',sep='\t'
#                            ,dtype={'citing_year':'object','citing_month':'object','citing_date':'object'}
                )

In [4]:
citation_net['time'] = pd.to_datetime(citation_net['time'],format="%Y-%m-%d")

In [5]:
def count_sub_df(sub_df):
    return pd.Series([sub_df['cited_uid'].values[0],sub_df.shape[0]],index=['uid','count'])

In [6]:
def month_rank_cited(year,month,df):
    higher_cited = df.loc[(df['time'].dt.month==int(month)) & (df['time'].dt.year==int(year))].groupby('cited_paper').apply(count_sub_df).sort_values('count',ascending=False)
    return higher_cited.reset_index()

In [7]:
def top_cited(rank_df):
    excess_one_count = rank_df.loc[rank_df['count']>=1].shape[0]
    print(f"excess_one_count {excess_one_count}")
    min_cited_cnt = rank_df.at[excess_one_count//100,'count']
    print(f"min_cited_cnt {min_cited_cnt}")
#     print(rank_df['count'])
    return rank_df.loc[rank_df['count']>=min_cited_cnt]

In [8]:
# 被引排序
cited_rank_each_month = {}
for year_month in ['2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10',
                 '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06'
             ]:
    print(year_month)
    year, month = year_month.split('-')
    cited_rank_paper = month_rank_cited(year,month,citation_net)
    cited_rank_each_month[year_month] = cited_rank_paper

2020-01
2020-02
2020-03
2020-04
2020-05
2020-06
2020-07
2020-08
2020-09
2020-10
2020-11
2020-12
2021-01
2021-02
2021-03
2021-04
2021-05
2021-06


In [9]:
# pmc_metadata = pd.read_csv('valid_metadata_0530.csv',sep='\t')
# pmc_metadata['publish_time'] = pd.to_datetime(pmc_metadata['publish_time'],format="%Y-%m-%d")

In [10]:
# for year_month in ['2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10',
#                  '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04'
#              ]:
#     year, month = year_month.split('-')
#     tmp_paper = pmc_metadata.loc[(pmc_metadata['publish_time'].dt.month==int(month)) & (pmc_metadata['publish_time'].dt.year==int(year))]
#     tmp_paper = tmp_paper[['title','cord_uid']]
#     tmp_paper = tmp_paper.rename(columns={'cord_uid':'uid','title':'cited_paper'})
#     tmp_paper['count'] = 0
#     tmp_paper = tmp_paper.loc[~(tmp_paper['uid'].isin(set(cited_rank_each_month[year_month]['uid'])))]
#     cited_rank_each_month[year_month] = pd.concat([cited_rank_each_month[year_month], tmp_paper],axis=0).reset_index(drop=True)

In [11]:
# number of being cited
for index,(year_month,df) in enumerate(cited_rank_each_month.items()):
    df.to_csv(f'cited_rank/cited_rank_paper_{year_month}_{data_dir}.tsv',sep='\t',index=False)

In [12]:
top_cited_each_month = {}
for year_month in ['2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10',
                 '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06'
             ]:
    print(year_month)
    tmp_top_paper = top_cited(cited_rank_each_month[year_month])
    top_cited_each_month[year_month] = tmp_top_paper

2020-01
excess_one_count 560
min_cited_cnt 3.0
2020-02
excess_one_count 523
min_cited_cnt 23.0
2020-03
excess_one_count 1650
min_cited_cnt 14.0
2020-04
excess_one_count 5942
min_cited_cnt 13.0
2020-05
excess_one_count 11023
min_cited_cnt 11.0
2020-06
excess_one_count 13806
min_cited_cnt 10.0
2020-07
excess_one_count 15007
min_cited_cnt 8.0
2020-08
excess_one_count 17940
min_cited_cnt 7.0
2020-09
excess_one_count 20125
min_cited_cnt 7.0
2020-10
excess_one_count 21906
min_cited_cnt 6.0
2020-11
excess_one_count 20601
min_cited_cnt 5.0
2020-12
excess_one_count 21784
min_cited_cnt 5.0
2021-01
excess_one_count 26271
min_cited_cnt 5.0
2021-02
excess_one_count 29886
min_cited_cnt 4.0
2021-03
excess_one_count 36723
min_cited_cnt 4.0
2021-04
excess_one_count 32427
min_cited_cnt 4.0
2021-05
excess_one_count 36587
min_cited_cnt 4.0
2021-06
excess_one_count 33592
min_cited_cnt 3.0


In [13]:
# save highly-impact papers
for year_month,df in top_cited_each_month.items():
    df.to_csv(f'top_cited/top_1_percente_{year_month}_{data_dir}.tsv',sep='\t',index=False)